In [1]:
import torch
print(torch.cuda.is_available())  # Should be True now
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4070 Laptop GPU


In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from sim_class import Simulation

class OT2Env(gym.Env):
    def __init__(self, render=False, max_steps=1000):
        super(OT2Env, self).__init__()
        self.render = render
        self.max_steps = max_steps

        # Create the simulation environment
        self.sim = Simulation(num_agents=1)

        # Define action and observation space
        # Action space: x, y, z velocities (each between -1 and 1)
        self.action_space = spaces.Box(
            low=-1, 
            high=1, 
            shape=(3,), 
            dtype=np.float32
        )
        
        # Observation space: [pipette_x, pipette_y, pipette_z, goal_x, goal_y, goal_z]
        self.observation_space = spaces.Box(
            low=-np.inf, 
            high=np.inf, 
            shape=(6,), 
            dtype=np.float32
        )

        # Define working area bounds (adjust these based on your OT-2 specs)
        self.x_bounds = (-0.187, 0.253)
        self.y_bounds = (-0.171, 0.220)
        self.z_bounds = (0.168, 0.290)

        # keep track of the number of steps
        self.steps = 0
        
        # Initialize goal position
        self.goal_position = None

    def _get_pipette_position(self, state):
        """Helper function to extract pipette position from state"""
        robot_key = [k for k in state.keys() if "robotId" in k][0]
        return np.array(state[robot_key]["pipette_position"], dtype=np.float32)

    def reset(self, seed=None, options=None):
        # being able to set a seed is required for reproducibility
        if seed is not None:
            np.random.seed(seed)
            # Also set the action space seed
            self.action_space.seed(seed)
            self.observation_space.seed(seed)

        # Reset the state of the environment to an initial state
        # set a random goal position for the agent
        self.goal_position = np.array([
            np.random.uniform(self.x_bounds[0], self.x_bounds[1]),
            np.random.uniform(self.y_bounds[0], self.y_bounds[1]),
            np.random.uniform(self.z_bounds[0], self.z_bounds[1])
        ], dtype=np.float32)
        
        # Call the environment reset function
        observation = self.sim.reset(num_agents=1)
        
        # Extract pipette position from observation using dynamic robot_id
        pipette_pos = self._get_pipette_position(observation)
        
        # Combine pipette position and goal position
        observation = np.concatenate([pipette_pos, self.goal_position])

        # Reset the number of steps
        self.steps = 0

        return observation, {}

    def step(self, action):
    # Execute one time step
        action_input = np.append(action, 0)
        observation = self.sim.run([action_input])
        pipette_pos = self._get_pipette_position(observation)
        observation = np.concatenate([pipette_pos, self.goal_position])

        if not hasattr(self, 'prev_distance'):
            self.prev_distance = np.linalg.norm(pipette_pos - self.goal_position)
        
        distance = np.linalg.norm(pipette_pos - self.goal_position)
        
        # Component 1: Progress reward more reward if you are closer to goal
        progress = (self.prev_distance - distance) * 10.0
        
        # Component 2: Distance-based reward the bigger the distance it gives less reward
        distance_reward = np.exp(-distance * 3.0)
        
        # Component 3: Action smoothness penalty (discourage jerky movements)
        action_penalty = -0.01 * np.linalg.norm(action)
        
        # Component 4: Time penalty the longer the model takes the more penalty
        time_penalty = -0.1
        
        # Combine all components
        reward = progress + distance_reward + action_penalty + time_penalty
        
        # Check completion
        # dont set trashhold to 0.0001 its not reachable
        threshold = 0.005
        if distance < threshold:
            terminated = True
            reward += 100.0
        else:
            terminated = False
        
        self.prev_distance = distance
        truncated = self.steps >= self.max_steps
        info = {'distance': float(distance), 'progress': float(progress)}
        self.steps += 1
        
        return observation, float(reward), terminated, truncated, info

    def render(self, mode='human'):
        pass
    
    def close(self):
        self.sim.close()

In [1]:
from stable_baselines3.common.env_checker import check_env
#from ot2_env_wrapper import OT2Env
import numpy as np

# First, check if the environment is valid
print("Checking environment...")
env = OT2Env()
check_env(env)
print("Environment check passed!")

# Now run some test episodes
print("\nRunning test episodes...")
num_episodes = 5

for episode in range(num_episodes):
    obs, info = env.reset()
    done = False
    step = 0
    total_reward = 0

    print(f"\n--- Episode {episode + 1} ---")
    print(f"Starting position: {obs[:3]}")
    print(f"Goal position: {obs[3:]}")

    while not done:
        # Take a random action from the environment's action space
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        
        total_reward += reward
        done = terminated or truncated

        step += 1
        
        if done:
            print(f"\nEpisode finished after {step} steps")
            print(f"Total reward: {total_reward:.2f}")
            print(f"Terminated: {terminated}, Truncated: {truncated}")
            break

env.close()
print("\nTesting complete!")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Checking environment...


NameError: name 'OT2Env' is not defined

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
#from ot2_env_wrapper import OT2Env
import os
import time
from datetime import datetime, timedelta

# Create directories
os.makedirs("models", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Create only ONE environment
env = OT2Env(render=False, max_steps=1000)

# Checkpoint callback only (no eval callback)
checkpoint_callback = CheckpointCallback(
    save_freq=10000,
    save_path="./models/",
    name_prefix="ot2_model"
)

# Create the PPO agent
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    tensorboard_log="./logs/tensorboard/",
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    device="cuda"
)

print("=" * 60)
print("Starting training...")
print(f"Total timesteps: 500,000")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

# Start timer
start_time = time.time()

# Train without eval callback
model.learn(
    total_timesteps=500000,
    callback=[checkpoint_callback],
    progress_bar=True
)

# End timer
end_time = time.time()
elapsed_time = end_time - start_time

# Convert to readable format
elapsed_td = timedelta(seconds=int(elapsed_time))
hours, remainder = divmod(int(elapsed_time), 3600)
minutes, seconds = divmod(remainder, 60)

print("=" * 60)
print("Training complete!")
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total training time: {hours}h {minutes}m {seconds}s")
print(f"Time per 1000 timesteps: {(elapsed_time / 500) * 1000:.2f} seconds")
print("=" * 60)

model.save("models/ot2_final_model")
print(f"Model saved to models/ot2_final_model.zip")

env.close()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Starting training...
Total timesteps: 500,000
Start time: 2025-12-10 11:31:26
Logging to ./logs/tensorboard/PPO_7


c:\Users\koenm\miniconda3\envs\drone_env\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Users\koenm\miniconda3\envs\drone_env\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | 441      |
| time/              |          |
|    fps             | 47       |
|    iterations      | 1        |
|    time_elapsed    | 43       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | 469         |
| time/                   |             |
|    fps                  | 43          |
|    iterations           | 2           |
|    time_elapsed         | 94          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005066338 |
|    clip_fraction        | 0.0188      |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.25       |
|    explained_variance   | -0.00991    |
|    learning_rate        | 0.